In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
import torch
import numpy as np
import accelerate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pathlib import Path

In [ ]:
# Navigate to parent directory
import os
os.chdir('../')
from src.data_processing import preprocess

In [ ]:
# Load and preprocess data
df = pd.read_csv('data/.csv')
df = df.dropna(subset=['Text', 'Rating'])
df['Text'] = df['Text'].apply(preprocess)

In [ ]:
# Create sentiment column

def map_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'
df['sentiment'] = df['Rating'].apply(map_sentiment)

In [ ]:
# Encode labels and split data
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'].tolist(), df['sentiment_encoded'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# Load BERT tokenizer and model using Auto classes
model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
# Format data to be used by PyTorch models

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [ ]:
# Calculate evaluation metrics

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Define training arguments

output_dir = Path('models/sentiment_model')

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Save the trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)